In [1]:
import pandas as pd
import numpy as np

df = pd.read_csv("original datasets.txt/players.txt")
print(df.shape)
df.head()

(3572, 11)


,ilkid,firstname,lastname,position,firstseason,lastseason,h_feet,h_inches,weight,college,birthdate
0,ABDELAL01,Alaa,Abdelnaby,F,1990,1994,6.0,10.0,240.0,Duke,1968-06-24 00:00:00
1,ABDULKA01,Kareem,Abdul-jabbar,C,1969,1988,7.0,2.0,225.0,UCLA,1947-04-16 00:00:00
2,ABDULMA01,Mahmo,Abdul-rauf,G,1990,2000,6.0,1.0,162.0,LSU,1969-03-09 00:00:00
3,ABDULTA01,Tariq,Abdul-wahad,G,1997,2002,6.0,6.0,223.0,Michigan,1974-11-03 00:00:00
4,ABDURSH01,Shareef,Abdur-rahim,F,1996,2004,6.0,9.0,225.0,California,1976-12-11 00:00:00


In [2]:
def fix_string_columns(df, columns):
    for column in columns:
        df[column] = df[column].astype('str')
        df[column] = df[column].str.strip()
        df[column] = df[column].str.lower()
    return df

In [3]:
df = fix_string_columns(df, ['ilkid', 'firstname', 'lastname', 'position', 'college'])

# check to see if there are any weird college labels
# df[df['college'].dropna().str.contains('\?|unk', regex=True)]

# standardize college labels
df.loc[:, 'college'] = df.college.fillna('unknown or none')
df.loc[df.college.isin(['?', 'nan', 'none?', 'none', 'Wooster ?']), 'college'] = 'unknown or none'

df['has_na_not_college'] = df.isnull().any(axis=1)

# convert birthdate to numeric (rough approximation)
df.loc[:, 'birthdate'] = pd.to_datetime(df.birthdate).apply(
    lambda x: np.round(365 * (x.year + x.month/12 + x.day/365)) / 365
)

# create a total height column
df['height'] = 12 * df.h_feet + df.h_inches
df['bmi'] = 703 * df.weight / df.height**2

# update birthdate, height, and weight missing values
# we are assuming they are correlated with first season and position played.
# to remove outliers we are taking the median values for a given first season and position played.
# then we average those two values to obtain the filler value.
columns_to_fill = ['birthdate', 'height', 'bmi']
columns_to_groupby = ['firstseason', 'position']
for col in columns_to_fill:
    df.loc[df[col].isnull(), col] = df[df[col].isnull()].join(
        df[[col] + columns_to_groupby].groupby(columns_to_groupby).median(),
        columns_to_groupby,
        lsuffix='dropme'
    ).drop([col + 'dropme'], axis=1)

# add a column for starting age (rounding to the nearest age, not the actual age).
# this will replace the birthdate column.
df['starting_age_nearest'] = (df.firstseason - df.birthdate).round().astype('int')
df = df.drop(['birthdate', 'h_feet', 'h_inches', 'weight'], axis=1)
assert len(df[df.isnull().any(axis=1)]) == 0, df[df.isnull().any(axis=1)]
df.to_csv('players.csv', index=False)
df.head()

,ilkid,firstname,lastname,position,firstseason,lastseason,college,has_na_not_college,height,bmi,starting_age_nearest
0,abdelal01,alaa,abdelnaby,f,1990,1994,duke,False,82.0,25.092207,21
1,abdulka01,kareem,abdul-jabbar,c,1969,1988,ucla,False,86.0,21.386560,22
2,abdulma01,mahmo,abdul-rauf,g,1990,2000,lsu,False,73.0,21.370989,21
3,abdulta01,tariq,abdul-wahad,g,1997,2002,michigan,False,78.0,25.767423,22
4,abdursh01,shareef,abdur-rahim,f,1996,2004,california,False,81.0,24.108368,19


In [4]:
regular_season = pd.read_csv('original datasets.txt/player_regular_season.txt')
playoffs = pd.read_csv('original datasets.txt/player_playoffs.txt')
allstar = pd.read_csv('original datasets.txt/player_allstar.txt')
regular_season['source'] = 'regular_season'
playoffs['source'] = 'playoffs'
# allstar['source'] = 'allstar'
# allstar = allstar.rename(columns={'conference': 'team'})
player = pd.concat([regular_season, playoffs])

# restrict our analysis to years when turnover was formally calculated
# see readme file for details, but we can use outlier detection as well
turnover_by_year = player.groupby(['year']).turnover.sum().reset_index()
turnover_by_year.turnover -= turnover_by_year.turnover.mean()
turnover_by_year.turnover /= turnover_by_year.turnover.abs().mean()
threshold = 1
turnover_by_year[turnover_by_year.turnover < -threshold]

,year,turnover
0,1946,-1.150799
1,1947,-1.150799
2,1948,-1.150799
3,1949,-1.150799
4,1950,-1.150799
5,1951,-1.150799
6,1952,-1.150799
7,1953,-1.150799
8,1954,-1.150799
9,1955,-1.150799


In [5]:
player = player[(player.year > 1970) & (player.year != 1976)]

player = fix_string_columns(player, ['ilkid', 'firstname', 'lastname'])

df = pd.merge(player, pd.read_csv('players.csv'), how='left', on='ilkid').sort_values('ilkid')
print(len(df)-len(df.drop_duplicates()))
df[df.isnull().any(axis=1)]

0


,ilkid,year,firstname_x,lastname_x,team,leag,gp,minutes,pts,oreb,...,firstname_y,lastname_y,position,firstseason,lastseason,college,has_na_not_college,height,bmi,starting_age_nearest


In [6]:
df.to_csv('player_combined.csv', index=False)

In [7]:
# checking the team stats file...
# it looks good.

df2 = pd.read_csv("original datasets.txt/team_season.txt")
print(len(df2)-len(df2.drop_duplicates()))
df2[df2.isnull().any(axis=1)]

0


,team,year,leag,o_fgm,o_fga,o_ftm,o_fta,o_oreb,o_dreb,o_reb,...,d_pf,d_stl,d_to,d_blk,d_3pm,d_3pa,d_pts,pace,won,lost


Missing values assumptions:
- birthdate, height and bmi are correllated with position and first year played
- first/last season are correlated with college
- we can lump all unknown college into a single category. "Unknown" and "none" are probably similar. If we don't know, they either probably didn't finish and/or didn't really care about college...